# Базовые скрипты для создания отчетов из Google Analytics через Analytics Reporting API версии 3

# ■ Получение информации об аккаунтах

In [1]:
# Пример: Вывод информации об ОДНОМ аккаунте
# https://developers.google.com/analytics/devguides/config/mgmt/v3/quickstart/service-py?hl=ru

# Пример 1: Запрашивает список всех учетных записей для авторизованного пользователя
# Пример 2: Результаты метода list сохраняются в объекте accounts. Код показывает, как перебирать их.
# https://developers.google.com/analytics/devguides/config/mgmt/v3/mgmtReference/management/accounts/list?hl=ru

# Аккаунты: метод list
# https://developers.google.com/analytics/devguides/config/mgmt/v3/mgmtReference/management/accounts/list?hl=ru

# Шаблон для объекта аккаунта:
# https://developers.google.com/analytics/devguides/config/mgmt/v3/mgmtReference/management/accounts?hl=ru#resource

In [2]:
# !pip install --upgrade google-api-python-client

In [3]:
import pandas as pd
import time
import sys
import socket

%pylab inline

# --- Google Analytics
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials


# При ошибках авторизации возвращается ошибка oauth2client.client.AccessTokenRefreshError, 
# которая свидетельствует о проблеме с токеном авторизации. 
# В этом случае приложение перенаправляет пользователя к процессу авторизации для получения нового токена.
from oauth2client.client import AccessTokenRefreshError

# При ошибках Management API возвращается ошибка apiclient.errors.HttpError, которая свидетельствует о проблемах 
# с доступом к API. В этом случае необходимо ознакомиться с сообщением об ошибке 
# и исправить процедуру доступа приложения к API.
from apiclient.errors import HttpError

# Вывод всех операций в Jupyter (а не только последней)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import logging
# Чтобы выводить на экран
logger = logging.getLogger()
logger.setLevel(logging.INFO)

Populating the interactive namespace from numpy and matplotlib


### Авторизация

In [4]:
# Имя файла-ключа
key_file_location = '1111111111111111111111111111111111111.json'

In [5]:
def get_service(api_name, api_version, scopes, key_file_location):
    """
    Получаем сервис (объект), который взаимодействует с Google API

    Аргументы:
        api_name: Имя API-сервиса
        api_version: Версия API
        scopes: A list auth scopes to authorize for the application.
        key_file_location: Имя JSON файла 

    Returns:
        Служба, подключенная к указанному API
    """

    credentials = ServiceAccountCredentials.from_json_keyfile_name(
            key_file_location, scopes=scopes)

    # Build the service object.
    service = build(api_name, api_version, credentials=credentials)

    return service

def authorization_api(key_file_location):
    """
    Авторизация     
    """
    scope = 'https://www.googleapis.com/auth/analytics.readonly'
    
    # Конструируем службу
    service = get_service(
            api_name='analytics',
            api_version='v3',
            scopes=[scope],
            key_file_location=key_file_location)
    
    return service

In [6]:
# Подсоединяемся к сервису
service = authorization_api(key_file_location)

### Запрашиваем список всех учетных записей

In [7]:
def get_accounts(service):    
    """
    Запрашиваем список всех учетных записей
    """
    try:
        accounts = service.management().accounts().list().execute()

    except TypeError as error:
      # Handle errors in constructing a query.
      print ('There was an error in constructing your query : %s' % error)

    except HttpError as error:
      # Handle API errors.
      print ('There was an API error : %s : %s' %
             (error.resp.status, error.resp.reason))
  
    # *** Перебираем объект "accounts" 
    for account in accounts.get('items', []):
        print( f"Account ID      = {account.get('id')}" )
        print( f"Account Name    = {account.get('name')}" )
        print( f"Created         = {account.get('created')}" )
        print( f"Updated         = {account.get('updated')}" )
        print( f"kind            = {account.get('kind')}" )
        
        # *** get profile_id
        account_id = account.get('id')
        # Get a list of all the properties for the first account.
        properties = service.management().webproperties().list(accountId=account_id).execute()

        if properties.get('items'):
            # Get the first property id.
            property = properties.get('items')[0].get('id')
            print( f"property        = {property}" )

        # Get a list of all views (profiles) for the first property.
        profiles = service.management().profiles().list(accountId=account_id,webPropertyId=property).execute()

        if profiles.get('items'):
            # return view (profile) id.
            print( f"view_id(первый) = {profiles.get('items')[0].get('id')}" )
        print('-----------------------------------------------')

# get_accounts(service)

### Запрашиваем список всех view_id у конкретного аккаунта

In [1]:
def get_view_id(service, account_id):    
    """
    Запрашиваем список всех view_id у конкретного аккаунта
    """
    # Get a list of all the properties for the first account.
    properties = service.management().webproperties().list(accountId=account_id).execute()

    if properties.get('items'):
        # Get the first property id.
        property = properties.get('items')[0].get('id')    
    
    try:
        # Получаем список профайлов (view_id)
        profiles = service.management().profiles().list(accountId=account_id,webPropertyId=property).execute()
    except TypeError as error:
        # Handle errors in constructing a query.
          print ('There was an error in constructing your query : %s' % error)
    except HttpError as error:
        # Handle API errors.
        print ('There was an API error : %s : %s' %(error.resp.status, error.resp.reason))
  
    # *** Перебираем объект "profiles" 
    print(f"--- account_id - {account_id} ---")
    for profile in profiles.get('items', []):
        print(f"{profile.get('id')} ----- {profile.get('name')}")

get_view_id(service, 121777428)

NameError: name 'service' is not defined

# ■ Список ресурсов, к которым у пользователей есть доступ

In [9]:
# https://developers.google.com/analytics/devguides/config/mgmt/v3/mgmtReference/management/webproperties/list?hl=ru
# https://developers.google.com/analytics/devguides/config/mgmt/v3/mgmtReference/management/webproperties?hl=ru#resource

In [10]:
def get_webproperties(service, accountId):
    try:
        properties = service.management().webproperties().list(accountId=accountId).execute()

    except TypeError as error:
      # Handle errors in constructing a query.
      print( 'There was an error in constructing your query : %s' % error )

    except HttpError as error:
      # Handle API errors.
      print ('There was an API error : %s : %s' % (error.resp.status, error.resp.reason) )
        
       
    # Извлекает все свойства для учетной записи пользователя, используя подстановочный знак "~" все в качестве идентификатора учетной записи.
    properties = service.management().webproperties().list(accountId='~all').execute()
    
        
    # Перебираем свойства     
    for property in properties.get('items', []):
        print( 'Account ID (whose resource)= %s' % property.get('accountId') )
        print( 'Property ID                = %s' % property.get('id') )
        print( 'Property Name              = %s' % property.get('name') )
        print( 'Property Profile Count     = %s' % property.get('profileCount') )
        print( 'Property Industry Vertical = %s' % property.get('industryVertical') )
        print( 'Property Internal Id       = %s' % property.get('internalWebPropertyId') )
        print( 'Property Level             = %s' % property.get('level') )
        if property.get('websiteUrl'):
            print( 'Property URL               = %s' % property.get('websiteUrl') )
            print( 'Created                    = %s' % property.get('created') )
            print( 'Updated                    = %s' % property.get('updated') )
        print()  

In [2]:
# Список аккаунтов        
list_accountId = [98397344, 171640763, 121777428] 

for accountId in list_accountId:
    print(f"\n*********  У пользователя (accountId: {accountId}) есть доступ к ресурсам:")
    get_webproperties(service, accountId)


*********  У пользователя (accountId: 98397344) есть доступ к ресурсам:


NameError: name 'get_webproperties' is not defined

# ■ Цели - List

In [12]:
# https://developers.google.com/analytics/devguides/config/mgmt/v3/mgmtReference/management/goals/list?hl=%%ruby

In [13]:
def get_list_goals(service, accountId, webPropertyId, profileId):
    # Example #4:
    # How to access a 'URL_DESTINATION' goals.
    def print_url_destination_goal_details(goal_details):
        print('\n\nGoal URL            = %s' % goal_details.get('url'))
        print('Case Sensitive      = %s' % goal_details.get('caseSensitive'))
        print('Match Type          = %s' % goal_details.get('matchType'))
        print('First Step Required = %s' % goal_details.get('firstStepRequired'))

        print('------ Url Destination Goal Steps -------')
        for goal_step in goal_details.get('steps', []):
            print('Step Number  = %s' % goal_step.get('number'))
            print('Step Name    = %s' % goal_step.get('name'))
            print('Step URL     = %s' % goal_step.get('url'))
        else:
            print('No Steps Configured')

        # Example #5:
        # How to access a 'VISIT_TIME_ON_SITE' goal.

    def print_visit_time_on_site_goal_details(goal_details):
        print('\n------ Visit Time On Site Goal -------')
        print('Comparison Type  = %s' % goal_details.get('comparisonType'))
        print('comparison Value = %s' % goal_details.get('comparisonValue'))

    # Example #6:
    # How to iterate through 'VISIT_NUM_PAGES' goals.
    def print_visit_num_pages_goal_details(goal_details):
        print('------ Visit Num Pages Goal -------')
        print('Comparison Type  = %s' % goal_details.get('comparisonType'))
        print('comparison Value = %s' % goal_details.get('comparisonValue'))

        # Example #7:
        # How to iterate through 'EVENT' goals.

    def print_event_goal_details(goal_details):
        print('Use Event Value  = %s' % goal_details.get('useEventValue'))

        for event_condition in goal_details.get('eventConditions', []):
            event_type = event_condition.get('type')
            print
            'Type             = %s' % event_type

            if event_type == 'VALUE':
                print('Comparison Type  = %s' % event_condition.get('comparisonType'))
                print('Comparison Value = %s' % event_condition.get('comparisonValue'))

            else:  # CATEGORY, ACTION, LABEL types.
                print('Match Type       = %s' % event_condition.get('matchType'))
                print('Expression       = %s' % event_condition.get('expression'))
    
# *********************************************************************************************************    
    # Example #1:
    # Запрашивает список всех целей для авторизованного пользователя
    try:
        goals = service.management().goals().list(
            accountId=accountId,
            webPropertyId=webPropertyId,
            profileId=profileId).execute()
    except TypeError as error:
        # Handle errors in constructing a query.
        print('There was an error in constructing your query : %s' % error)

    except HttpError as error:
        # Handle API errors.
        print('There was an API error : %s : %s' %
              (error.resp.status, error.resp.reason()))

    # Example #2:
    # Извлекает все цели для КОНКРЕТНОГО представления пользователя, (если необходимо брать все profileId - используем
    # подстановочный знак '~all' в качестве идентификатора профиля).
    goals = service.management().goals().list(accountId=accountId,
                                                webPropertyId=webPropertyId,
                                                profileId=profileId).execute()

    # Example #3:
    # Перебираем цели
    for goal in goals.get('items', []):
        print('Account ID           = %s' % goal.get('accountId'))
        print('Property ID          = %s' % goal.get('webPropertyId'))
        print('Internal Property ID = %s' % goal.get('internalWebPropertyId'))
        print('View (Profile) ID    = %s' % goal.get('profileId'))

        print('Goal Number = %s' % goal.get('id'))
        print('Goal Name   = %s' % goal.get('name'))
        print('Goal Value  = %s' % goal.get('value'))
        print('Goal Active = %s' % goal.get('active'))
        print('Goal Type   = %s' % goal.get('type'))

        print
        'Created     = %s' % goal.get('created')
        print
        'Updated     = %s' % goal.get('updated')

        # Распечатайте сведения о цели в зависимости от типа цели
        if goal.get('urlDestinationDetails'):
            print_url_destination_goal_details(goal.get('urlDestinationDetails'))

        elif goal.get('visitTimeOnSiteDetails'):
            print_visit_time_on_site_goal_details(goal.get('visitTimeOnSiteDetails'))

        elif goal.get('visitNumPagesDetails'):
            print_visit_num_pages_goal_details(goal.get('visitNumPagesDetails'))

        elif goal.get('eventDetails'):
            print_event_goal_details(goal.get('eventDetails'))  

In [3]:
accountId = 11111111
webPropertyId = 'UA-44444444-1'
view_id = "ga:"+"22222222"
profileId = '33333333'

get_list_goals(service, accountId, webPropertyId, profileId)

NameError: name 'get_list_goals' is not defined

# ■ "Core Reporting API" - пример запроса

In [15]:
# https://developers.google.com/analytics/devguides/reporting/core/v3/coreDevguide?hl=ru#python
# https://github.com/googleapis/google-api-python-client/blob/cbb1f88b82b21f5cb9dcace33ffea3f95a189015/samples/analytics/core_reporting_v3_reference.py

In [16]:
def get_api_query(service, **kwargs):
    """
    Возвращает объет запроса
    Args:
    service: The service object built by the Google API Python client library.
    """
    # --- Эскпоненциальная задержка ---------------------------
    try:
        logging.info("запуск - query.execute()")
        query = service.data().ga().get(**kwargs)
        # results = query.execute()
        # return results
        return query.execute()
    except HttpError as error:
        # https://developers.google.com/analytics/devguides/reporting/mcf/v3/mcfErrors?hl=ru
        # В документации reason (причину) получаем через error.resp.reason. Но это сейчас не работает.
        # Вручную получаем через error
        # print(f"error.resp --- {reason}")
        # reason = reason.split("reason")[1]
        # reason = reason.split(":")[1]
        # reason = reason.split("'")[1]

        logging.info(f"Эскпоненциальная задержка: Запрос пока не прошел. Ответ GA: {error.resp.status} --- {error.resp.reason} --- {error._get_reason()}")
        # if reason in ['userRateLimitExceeded', 'quotaExceeded',
        #                          'internalServerError', 'backendError']:
        if error.resp.status in [403, 500, 503]:
            logging.info("Эскпоненциальная задержка: код указанной ошибке позволяет запустить повтор...")
            t = 0
            expontential_backoff = 2
            while t < 10:
                logging.info(f"Эскпоненциальная задержка: ЗАДЕРЖКА - {2 ** expontential_backoff} сек")
                time.sleep(2 ** expontential_backoff)
                try:
                    return  query.execute()
                except HttpError as error:
                    logging.info(f"Эскпоненциальная задержка: Запрос пока не прошел. Ответ GA: {error.resp.status} --- {error.resp.reason}")
                    expontential_backoff += 1
                    t += 1
                    logging.info(f"Эскпоненциальная задержка: Увеличен период экспоненциальной задержки до {2 ** expontential_backoff} сек., попытка №{t}")
                    if t >= 10:
                        logging.info(f"Было предпринято {t} попыток сделать запрос. Все попытки не помогли. Возможно перегружен сервер. Необходимо повторить позже.")
                        return False
                    else:
                        continue
        else:
            logging.error(f"Запрос не прошел. Ошибка: {error} ---{error.resp}----- {error.resp.reason}")
            return False
    
#     except TypeError as error:
#         # Handle errors in constructing a query.
#         print(('There was an error in constructing your query : %s' % error))
#         return False

#     except AccessTokenRefreshError:
#         # Handle Auth errors.
#         print ('The credentials have been revoked or expired, please re-run ',  'the application to re-authorize')
#         return False

In [17]:
def print_report_info(results):
    """
    Общая информация об отчете
    Args:
    results: The response returned from the Core Reporting API.
    """

    print('Информация об отчете:')
    print('Contains Sampled Data (выборка есть?) = %s' % results.get('containsSampledData'))
    print('Kind                  = %s' % results.get('kind'))
    print('ID                    = %s' % results.get('id'))
    print('Self Link             = %s' % results.get('selfLink'))
    print()


def print_pagination_info(results):
    """
    Prints common pagination details.
    Args:
    results: The response returned from the Core Reporting API.
    """

    print('\nРазбиение на страницы:')
    print('Items per page = %s' % results.get('itemsPerPage'))
    print('Total Results  = %s' % results.get('totalResults'))

    # Только, если есть другие страницы результатов
    if results.get('previousLink'):
        print('Previous Link  = %s' % results.get('previousLink'))
    if results.get('nextLink'):
        print('Next Link      = %s' % results.get('nextLink'))

def print_sample_Size_Space(results):
    """
    выводим параметры семплирования
    """

    print('\nПараметры семплирования:')
    if (results.get('containsSampledData')):
        print("Семплирование (ВЫБОРКА) - есть!!!")
        print('sampleSize (Количество строк, используемое для выборки) = %s' % results.get('sampleSize'))
        print('sampleSpace (Общий размер выборочного пространства, из которого отбираются строки) = %s' % results.get('sampleSpace'))
        
        
def print_profile_info(results):
    """
    Prints information about the profile.
    Args:
    results: The response returned from the Core Reporting API.
    """

    print('\nИнформация об аккаунте:')
    info = results.get('profileInfo')
    print('Account Id      = %s' % info.get('accountId'))
    print('Web Property Id = %s' % info.get('webPropertyId'))
    print('Profile Id      = %s' % info.get('profileId'))
    print('Table Id        = %s' % info.get('tableId'))
    print('Profile Name    = %s' % info.get('profileName'))


def print_query(results):
    """
    The query returns the original report query as a dict.
    Args:
    results: The response returned from the Core Reporting API.
    """

    print('\nПараметры запроса:')
    query = results.get('query')
    for key, value in query.items():
        print('%s = %s' % (key, value))
    print()

def print_column_headers(results):
    """
    Prints the information for each column.
    The main data from the API is returned as rows of data. The column
    headers describe the names and types of each column in rows.
    Args:
    results: The response returned from the Core Reporting API.
    """

    print('\nColumn Headers:')
    headers = results.get('columnHeaders')
    for header in headers:
        # Print Dimension or Metric name.
        print('\t%s name:    = %s' % (header.get('columnType').title(),
                                      header.get('name')))
        print('\tColumn Type = %s' % header.get('columnType'))
        print('\tData Type   = %s' % header.get('dataType'))
        print()


def print_totals_for_all_results(results):
    """
    Prints the total metric value for all pages the query matched.
    Args:
    results: The response returned from the Core Reporting API.
    """

    print('\nTotal Metrics For All Results:')
    print('This query returned %s rows.' % len(results.get('rows')))
    print(('But the query matched %s total results.' %
         results.get('totalResults')))
    print('Ниже приведены итоговые показатели для сопоставленных итоговых результатов.')
    totals = results.get('totalsForAllResults')

    for metric_name, metric_total in totals.items():
        print('Metric Name  = %s' % metric_name)
        print('Metric Total = %s' % metric_total)
        print()

def result_one_page_to_dataframe(results):
    """
    Результат запроса - в датафрейм, НО толька ПЕРВАЯ СТРАНИЦА
    На входе - результат запроса
    """
    if results.get('rows', []):
        headers = results.get('columnHeaders')
        columns = [el['name'][3:]  for el in headers]
        data=results.get('rows')
        df = pd.DataFrame(data=data, columns=columns)
        return df        
    else:
        print('No Rows Found')

        
def import_from_GA_raw_to_dataframe(service, view_id, **params):
    """
    Получение результата запроса в датафрейме
    В GA есть ограничение - 10000 строк (одна страница запроса). 
    Поэтому страницы склеиваются в один датафрейм
    
    Официальная документация: "Информация о разбиении на страницы"
    https://developers.google.com/analytics/devguides/reporting/core/v3/coreDevguide?hl=ru
    https://stackoverflow.com/questions/11934418/using-nextlink-attribute-to-get-the-next-result-page
    """
        
    has_more = True
    df = pd.DataFrame()

    while has_more:
        results = get_api_query(service, **params)

        # Итоговый датафрейм
        df_plus = result_one_page_to_dataframe(results)
        df = pd.concat([df, df_plus], axis=0 )

        # Увеличиваем начальный индекс     
        params['start_index'] = int(params['start_index']) + int(params['max_results'])
        has_more = results.get('nextLink')
    
    return df

# Пример запроса

In [18]:
view_id = "ga:"+"11111111111"   

In [4]:
params = {
    'ids':view_id,
    'start_date':'2022-05-13',
    'end_date':'2022-05-13',
    'dimensions':'ga:datehourminute, ga:clientid',
    'metrics':'ga:pageviews',
#     'sort':'-ga:visits',
#     'filters':'ga:medium==organic',
    'start_index':1,
    'max_results':10000,
    'samplingLevel': 'HIGHER_PRECISION'
}    

results = get_api_query(service, **params)

# Распечатываем результаты запроса
print_report_info(results)
print_sample_Size_Space(results)
print_pagination_info(results)
print_profile_info(results)
print_query(results)
print_column_headers(results)
print_totals_for_all_results(results)

NameError: name 'get_api_query' is not defined

In [20]:
# Устанавливаем предельное время запроса. Если этого не сделать не будет срабатывать "except HttpError as error", т.е. не будет олавливать ошибки запроса
TIMEOUT = 1200   # 20 мин
socket.setdefaulttimeout(TIMEOUT)

# Итоговый датафрейм
df = import_from_GA_raw_to_dataframe(service, view_id, **params)
df

INFO:root:запуск - query.execute()
INFO:root:запуск - query.execute()
INFO:root:запуск - query.execute()
INFO:root:запуск - query.execute()
INFO:root:запуск - query.execute()
INFO:root:запуск - query.execute()
INFO:root:запуск - query.execute()
INFO:root:запуск - query.execute()
INFO:root:запуск - query.execute()
INFO:root:запуск - query.execute()
INFO:root:запуск - query.execute()
INFO:root:запуск - query.execute()
INFO:root:запуск - query.execute()
INFO:root:запуск - query.execute()


,datehourminute,clientid,pageviews
0,202205130000,11892722.1649411145,1
1,202205130000,528876029.1635799078,1
2,202205130000,356059219.1652389170,1
3,202205130000,174263701.1629585014,1
4,202205130000,1803283526.1652389226,5
...,...,...,...
8127,202205132359,2028347019.1652475323,1
8128,202205132359,1126058694.1648586613,1
8129,202205132359,952676248.1644945733,1
8130,202205132359,1056161027.1652458673,4
